In [64]:
# import library
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import streamlit as st
from pandas.tseries.offsets import DateOffset

In [65]:
# define functions

# load data
df_vid = pd.read_csv('resource/Video_Performance_Over_Time.csv')

# dropping row Total to avoid issue with calculations
df_agg = pd.read_csv('resource/Aggregated_Metrics_By_Video.csv').iloc[1:,:]

df_agg_sub = pd.read_csv('resource/Aggregated_Metrics_By_Country_And_Subscriber_Status.csv')

df_com = pd.read_csv('resource/All_Comments_Final.csv')

In [66]:
# clean our data

In [67]:
# converting all columns to uppercase
for df in [df_agg,df_vid,df_agg_sub, df_com]:
    df.columns = df.columns.str.upper()


In [68]:
# checking
df_com.columns

Index(['COMMENTS', 'COMMENT_ID', 'REPLY_COUNT', 'LIKE_COUNT', 'DATE', 'VIDID',
       'USER_ID'],
      dtype='object')

In [69]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 1 to 223
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   VIDEO                                 223 non-null    object 
 1   VIDEO TITLE                           223 non-null    object 
 2   VIDEO PUB­LISH TIME                   223 non-null    object 
 3   COM­MENTS AD­DED                      223 non-null    int64  
 4   SHARES                                223 non-null    int64  
 5   DIS­LIKES                             223 non-null    int64  
 6   LIKES                                 223 non-null    int64  
 7   SUB­SCRIBERS LOST                     223 non-null    int64  
 8   SUB­SCRIBERS GAINED                   223 non-null    int64  
 9   RPM (USD)                             223 non-null    float64
 10  CPM (USD)                             221 non-null    float64
 11  AV­ER­AGE PER­CENT­

In [70]:
df_agg.head(1)

,VIDEO,VIDEO TITLE,VIDEO PUB­LISH TIME,COM­MENTS AD­DED,SHARES,DIS­LIKES,LIKES,SUB­SCRIBERS LOST,SUB­SCRIBERS GAINED,RPM (USD),CPM (USD),AV­ER­AGE PER­CENT­AGE VIEWED (%),AV­ER­AGE VIEW DUR­A­TION,VIEWS,WATCH TIME (HOURS),SUB­SCRIBERS,YOUR ES­TIM­ATED REV­EN­UE (USD),IM­PRES­SIONS,IM­PRES­SIONS CLICK-THROUGH RATE (%)
1,4OZip0cgOho,How I Would Learn Data Science (If I Had to St...,"May 8, 2020",907,9583,942,46903,451,46904,6.353,12.835,36.65,0:03:09,1253559,65850.7042,46453,7959.533,26498799,3.14


In [71]:
# checking columns
df_agg.columns.tolist() # notice \xad in our columns

['VIDEO',
 'VIDEO TITLE',
 'VIDEO PUB\xadLISH TIME',
 'COM\xadMENTS AD\xadDED',
 'SHARES',
 'DIS\xadLIKES',
 'LIKES',
 'SUB\xadSCRIBERS LOST',
 'SUB\xadSCRIBERS GAINED',
 'RPM (USD)',
 'CPM (USD)',
 'AV\xadER\xadAGE PER\xadCENT\xadAGE VIEWED (%)',
 'AV\xadER\xadAGE VIEW DUR\xadA\xadTION',
 'VIEWS',
 'WATCH TIME (HOURS)',
 'SUB\xadSCRIBERS',
 'YOUR ES\xadTIM\xadATED REV\xadEN\xadUE (USD)',
 'IM\xadPRES\xadSIONS',
 'IM\xadPRES\xadSIONS CLICK-THROUGH RATE (%)']

In [72]:
# remove \xad
df_agg.columns = df_agg.columns.str.replace('\xad','')

In [73]:
# check 
df_agg.columns.tolist()

['VIDEO',
 'VIDEO TITLE',
 'VIDEO PUBLISH TIME',
 'COMMENTS ADDED',
 'SHARES',
 'DISLIKES',
 'LIKES',
 'SUBSCRIBERS LOST',
 'SUBSCRIBERS GAINED',
 'RPM (USD)',
 'CPM (USD)',
 'AVERAGE PERCENTAGE VIEWED (%)',
 'AVERAGE VIEW DURATION',
 'VIEWS',
 'WATCH TIME (HOURS)',
 'SUBSCRIBERS',
 'YOUR ESTIMATED REVENUE (USD)',
 'IMPRESSIONS',
 'IMPRESSIONS CLICK-THROUGH RATE (%)']

In [74]:
# convert date to datetype

df_agg['VIDEO PUBLISH TIME'] = pd.to_datetime(df_agg['VIDEO PUBLISH TIME'], format='mixed')

In [75]:
df_agg['VIDEO PUBLISH TIME']

1     2020-05-08
2     2020-11-12
3     2020-07-16
4     2020-08-29
5     2020-08-05
         ...    
219   2018-11-30
220   2019-05-25
221   2018-12-18
222   2019-05-05
223   2017-06-06
Name: VIDEO PUBLISH TIME, Length: 223, dtype: datetime64[ns]

In [76]:
df_agg['VIDEO PUBLISH TIME'][9]

Timestamp('2020-07-01 00:00:00')

In [77]:
df_agg.head(1)

,VIDEO,VIDEO TITLE,VIDEO PUBLISH TIME,COMMENTS ADDED,SHARES,DISLIKES,LIKES,SUBSCRIBERS LOST,SUBSCRIBERS GAINED,RPM (USD),CPM (USD),AVERAGE PERCENTAGE VIEWED (%),AVERAGE VIEW DURATION,VIEWS,WATCH TIME (HOURS),SUBSCRIBERS,YOUR ESTIMATED REVENUE (USD),IMPRESSIONS,IMPRESSIONS CLICK-THROUGH RATE (%)
1,4OZip0cgOho,How I Would Learn Data Science (If I Had to St...,2020-05-08,907,9583,942,46903,451,46904,6.353,12.835,36.65,0:03:09,1253559,65850.7042,46453,7959.533,26498799,3.14


In [78]:
# AVERAGE VIEW DURATION is an object meaning it contains numbers and string
df_agg['AVERAGE VIEW DURATION'][9] # a string

'0:04:56'

In [79]:
df_agg['AVERAGE VIEW DURATION'] = pd.to_datetime(df_agg['AVERAGE VIEW DURATION']).dt.time


# check
df_agg['AVERAGE VIEW DURATION'][9]

/tmp/ipykernel_55684/2397845352.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_agg['AVERAGE VIEW DURATION'] = pd.to_datetime(df_agg['AVERAGE VIEW DURATION']).dt.time


datetime.time(0, 4, 56)

In [80]:
# create new column for df_agg['AVERAGE VIEW SECONDS'
df_agg['AVERAGE VIEW SECONDS'] = df_agg['AVERAGE VIEW DURATION'].apply(lambda x: x.second + x.minute * 60 + x.hour * 60 * 60)

# check
df_agg['AVERAGE VIEW SECONDS'][9]

296

In [81]:
df_agg.head(1)

,VIDEO,VIDEO TITLE,VIDEO PUBLISH TIME,COMMENTS ADDED,SHARES,DISLIKES,LIKES,SUBSCRIBERS LOST,SUBSCRIBERS GAINED,RPM (USD),CPM (USD),AVERAGE PERCENTAGE VIEWED (%),AVERAGE VIEW DURATION,VIEWS,WATCH TIME (HOURS),SUBSCRIBERS,YOUR ESTIMATED REVENUE (USD),IMPRESSIONS,IMPRESSIONS CLICK-THROUGH RATE (%),AVERAGE VIEW SECONDS
1,4OZip0cgOho,How I Would Learn Data Science (If I Had to St...,2020-05-08,907,9583,942,46903,451,46904,6.353,12.835,36.65,00:03:09,1253559,65850.7042,46453,7959.533,26498799,3.14,189


In [82]:
df_agg.info() # our data type is corrected

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 1 to 223
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   VIDEO                               223 non-null    object        
 1   VIDEO TITLE                         223 non-null    object        
 2   VIDEO PUBLISH TIME                  223 non-null    datetime64[ns]
 3   COMMENTS ADDED                      223 non-null    int64         
 4   SHARES                              223 non-null    int64         
 5   DISLIKES                            223 non-null    int64         
 6   LIKES                               223 non-null    int64         
 7   SUBSCRIBERS LOST                    223 non-null    int64         
 8   SUBSCRIBERS GAINED                  223 non-null    int64         
 9   RPM (USD)                           223 non-null    float64       
 10  CPM (USD)                 

In [83]:
# engagement ration, every engagement a view could do divided by the number of viewrs
df_agg['ENGAGEMENT RATIO'] = (df_agg['SHARES'] + df_agg['LIKES'] + df_agg['DISLIKES'] + df_agg['COMMENTS ADDED']) / df_agg['VIEWS'] 

# ratio of views to subscribers gained
df_agg['VIEW TO SUBSCRIBER RATIO'] = df_agg['VIEWS'] / df_agg['SUBSCRIBERS GAINED'] # how views does it take to gain a subscriber

# ratio of views to subscribers lost, 
df_agg['VIEW TO SUBSCRIBER LOST RATIO'] = df_agg['VIEWS'] / df_agg['SUBSCRIBERS LOST'] # how views does it take to lose a subscriber

In [84]:
df_agg.head(1)

,VIDEO,VIDEO TITLE,VIDEO PUBLISH TIME,COMMENTS ADDED,SHARES,DISLIKES,LIKES,SUBSCRIBERS LOST,SUBSCRIBERS GAINED,RPM (USD),...,VIEWS,WATCH TIME (HOURS),SUBSCRIBERS,YOUR ESTIMATED REVENUE (USD),IMPRESSIONS,IMPRESSIONS CLICK-THROUGH RATE (%),AVERAGE VIEW SECONDS,ENGAGEMENT RATIO,VIEW TO SUBSCRIBER RATIO,VIEW TO SUBSCRIBER LOST RATIO
1,4OZip0cgOho,How I Would Learn Data Science (If I Had to St...,2020-05-08,907,9583,942,46903,451,46904,6.353,...,1253559,65850.7042,46453,7959.533,26498799,3.14,189,0.046536,26.726057,2779.509978


In [85]:
# sort data by 'VIDEO PUBLISH TIME'
df_agg.sort_values(by = 'VIDEO PUBLISH TIME', ascending=False, inplace=True)

df_agg = df_agg.reset_index(drop=True)

In [86]:
df_agg.head(2)

,VIDEO,VIDEO TITLE,VIDEO PUBLISH TIME,COMMENTS ADDED,SHARES,DISLIKES,LIKES,SUBSCRIBERS LOST,SUBSCRIBERS GAINED,RPM (USD),...,VIEWS,WATCH TIME (HOURS),SUBSCRIBERS,YOUR ESTIMATED REVENUE (USD),IMPRESSIONS,IMPRESSIONS CLICK-THROUGH RATE (%),AVERAGE VIEW SECONDS,ENGAGEMENT RATIO,VIEW TO SUBSCRIBER RATIO,VIEW TO SUBSCRIBER LOST RATIO
0,0jTtHYie3CU,Should You Be Excited About Web 3? (As a Data ...,2022-01-17,37,43,8,267,14,18,4.055,...,4383,192.5779,4,16.549,65130,2.95,158,0.080995,243.5,313.071429
1,2RWwN5ZT4tA,Should @Luke Barousse Take This Data Analyst ...,2022-01-14,12,2,3,78,1,1,1.882,...,2401,25.9375,0,1.720,25094,2.64,38,0.039567,2401.0,2401.000000


In [87]:
df_vid.head(1)

,DATE,VIDEO TITLE,EXTERNAL VIDEO ID,VIDEO LENGTH,THUMBNAIL LINK,VIEWS,VIDEO LIKES ADDED,VIDEO DISLIKES ADDED,VIDEO LIKES REMOVED,USER SUBSCRIPTIONS ADDED,USER SUBSCRIPTIONS REMOVED,AVERAGE VIEW PERCENTAGE,AVERAGE WATCH TIME,USER COMMENTS ADDED
0,19 Jan 2022,Kaggle Project From Scratch - Part 2 (Explorat...,KQ80oD_boBM,2191,https://i.ytimg.com/vi/KQ80oD_boBM/hqdefault.jpg,13,0,0,0,0,0,0.069055,151.300154,0


In [88]:
# CONVERTING DATE to datetime
df_vid['DATE'] = pd.to_datetime(df_vid['DATE'], format='mixed').dt.date

In [89]:
df_agg_sub.head(1)

,VIDEO TITLE,EXTERNAL VIDEO ID,VIDEO LENGTH,THUMBNAIL LINK,COUNTRY CODE,IS SUBSCRIBED,VIEWS,VIDEO LIKES ADDED,VIDEO DISLIKES ADDED,VIDEO LIKES REMOVED,USER SUBSCRIPTIONS ADDED,USER SUBSCRIPTIONS REMOVED,AVERAGE VIEW PERCENTAGE,AVERAGE WATCH TIME,USER COMMENTS ADDED
0,🌶 Hot Topics in Tech: Data Science Explained #...,OtqQYqRNDGI,59,https://i.ytimg.com/vi/OtqQYqRNDGI/hqdefault.jpg,HK,True,23,1,0,0,2,0,0.67187,39.640348,0


In [90]:
df_com.head(1)

,COMMENTS,COMMENT_ID,REPLY_COUNT,LIKE_COUNT,DATE,VIDID,USER_ID
0,Thanks for this video Ken.\n\nI decided to go ...,UgxFZTIzC4UPyhhX_PZ4AaABAg,0,0,2022-01-22T08:13:29Z,xpIFS6jZbe8,user_981


In [91]:
df_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   COMMENTS     10239 non-null  object
 1   COMMENT_ID   10240 non-null  object
 2   REPLY_COUNT  10240 non-null  int64 
 3   LIKE_COUNT   10240 non-null  int64 
 4   DATE         10240 non-null  object
 5   VIDID        10240 non-null  object
 6   USER_ID      10240 non-null  object
dtypes: int64(2), object(5)
memory usage: 560.1+ KB


In [92]:
df_com['DATE'] = pd.to_datetime(df_com['DATE']).dt.date

In [93]:
df_com.head(1)

,COMMENTS,COMMENT_ID,REPLY_COUNT,LIKE_COUNT,DATE,VIDID,USER_ID
0,Thanks for this video Ken.\n\nI decided to go ...,UgxFZTIzC4UPyhhX_PZ4AaABAg,0,0,2022-01-22,xpIFS6jZbe8,user_981


In [94]:
df_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   COMMENTS     10239 non-null  object
 1   COMMENT_ID   10240 non-null  object
 2   REPLY_COUNT  10240 non-null  int64 
 3   LIKE_COUNT   10240 non-null  int64 
 4   DATE         10240 non-null  object
 5   VIDID        10240 non-null  object
 6   USER_ID      10240 non-null  object
dtypes: int64(2), object(5)
memory usage: 560.1+ KB


In [95]:
# engineer data
## what metrics wil be relevant
## difference from baseline
## percent change

# build dashboard
## local picture
## individual video

# improvement

In [96]:
def load_data():

    # load data
    df_vid = pd.read_csv('resource/Video_Performance_Over_Time.csv')

    # dropping row Total to avoid issue with calculations
    df_agg = pd.read_csv('resource/Aggregated_Metrics_By_Video.csv').iloc[1:,:]

    df_agg_sub = pd.read_csv('resource/Aggregated_Metrics_By_Country_And_Subscriber_Status.csv')

    df_com = pd.read_csv('resource/All_Comments_Final.csv')

    # converting all columns to uppercase
    for df in [df_agg,df_vid,df_agg_sub, df_com]:
        df.columns = df.columns.str.upper()
        
    # remove \xad
    df_agg.columns = df_agg.columns.str.replace('\xad','')
    # convert date to datetype

    df_agg['VIDEO PUBLISH TIME'] = pd.to_datetime(df_agg['VIDEO PUBLISH TIME'], format='mixed')

    df_agg['AVERAGE VIEW DURATION'] = df_agg['AVERAGE VIEW DURATION'].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))

    # create new column for df_agg['AVERAGE VIEW SECONDS'
    df_agg['AVERAGE VIEW SECONDS'] = df_agg['AVERAGE VIEW DURATION'].apply(lambda x: x.second + x.minute * 60 + x.hour * 60 * 60)

    # engagement ration, every engagement a view could do divided by the number of viewrs
    df_agg['ENGAGEMENT RATIO'] = (df_agg['SHARES'] + df_agg['LIKES'] + df_agg['DISLIKES'] + df_agg['COMMENTS ADDED']) / df_agg['VIEWS'] 

    # ratio of views to subscribers gained
    df_agg['VIEW TO SUBSCRIBER RATIO'] = df_agg['VIEWS'] / df_agg['SUBSCRIBERS GAINED'] # how views does it take to gain a subscriber

    # ratio of views to subscribers lost, 
    df_agg['VIEW TO SUBSCRIBER LOST RATIO'] = df_agg['VIEWS'] / df_agg['SUBSCRIBERS LOST'] # how views does it take to lose a subscriber

    # sort data by 'VIDEO PUBLISH TIME'
    df_agg.sort_values(by = 'VIDEO PUBLISH TIME', ascending=False, inplace=True)

    # CONVERTING DATE to datetime
    df_vid['DATE'] = pd.to_datetime(df_vid['DATE'], format='mixed')

    df_com['DATE'] = pd.to_datetime(df_com['DATE'])
    
    return df_vid, df_agg, df_agg_sub, df_com

In [97]:
    
df_vid, df_agg, df_agg_sub, df_com = load_data()

In [98]:
# aggregated differential 

# create a copy of our dataframe
df_agg_diff = df_agg.copy()

In [99]:
# for the last 12 months, most recently date back to 12 months
metric_date_12mo = df_agg_diff['VIDEO PUBLISH TIME'].max() - DateOffset(months=12)
metric_date_12mo # 12 months early date

Timestamp('2021-01-17 00:00:00')

In [100]:
# dataframe from metric_date_12mo to df_agg_diff['VIDEO PUBLISH TIME'].max()
# that is, from 12 monts early to current date

df_agg_diff_12mo = df_agg_diff[df_agg_diff['VIDEO PUBLISH TIME'] >= metric_date_12mo] # we have 48 rows
df_agg_diff_12mo.shape

(48, 23)

In [101]:
median_agg = df_agg_diff_12mo[df_agg_diff_12mo.columns[2:]].median()
median_agg

VIDEO PUBLISH TIME                    2021-07-06 00:00:00
COMMENTS ADDED                                       43.5
SHARES                                               42.5
DISLIKES                                              5.0
LIKES                                               382.5
SUBSCRIBERS LOST                                     13.0
SUBSCRIBERS GAINED                                   56.5
RPM (USD)                                            4.37
CPM (USD)                                          10.573
AVERAGE PERCENTAGE VIEWED (%)                      41.175
AVERAGE VIEW DURATION                 1900-01-01 00:02:46
VIEWS                                              7417.0
WATCH TIME (HOURS)                               279.9851
SUBSCRIBERS                                          38.5
YOUR ESTIMATED REVENUE (USD)                      24.7995
IMPRESSIONS                                      155102.5
IMPRESSIONS CLICK-THROUGH RATE (%)                   2.43
AVERAGE VIEW S

In [102]:
# aggregated differential 

# create a copy of our dataframe
df_agg_diff = df_agg.copy()

In [103]:
df_agg['VIDEO PUBLISH TIME']

111   2022-01-17
187   2022-01-14
64    2022-01-10
59    2022-01-03
32    2021-12-27
         ...    
75    2018-11-14
190   2018-11-12
204   2018-07-10
138   2017-11-18
223   2017-06-06
Name: VIDEO PUBLISH TIME, Length: 223, dtype: datetime64[ns]

In [104]:
# for the last 12 months, most recently date back to 12 months
metric_date_12mo = df_agg_diff['VIDEO PUBLISH TIME'].max() - DateOffset(months=12)

metric_date_12mo = metric_date_12mo

# dataframe from metric_date_12mo to df_agg_diff['VIDEO PUBLISH TIME'].max()
# that is, from 12 months early to current date

df_agg_diff_12mo = df_agg_diff[df_agg_diff['VIDEO PUBLISH TIME'] >= metric_date_12mo]
# median 
median_agg = df_agg_diff_12mo[df_agg_diff_12mo.columns[2:]].median()

In [105]:
median_agg

VIDEO PUBLISH TIME                    2021-07-06 00:00:00
COMMENTS ADDED                                       43.5
SHARES                                               42.5
DISLIKES                                              5.0
LIKES                                               382.5
SUBSCRIBERS LOST                                     13.0
SUBSCRIBERS GAINED                                   56.5
RPM (USD)                                            4.37
CPM (USD)                                          10.573
AVERAGE PERCENTAGE VIEWED (%)                      41.175
AVERAGE VIEW DURATION                 1900-01-01 00:02:46
VIEWS                                              7417.0
WATCH TIME (HOURS)                               279.9851
SUBSCRIBERS                                          38.5
YOUR ESTIMATED REVENUE (USD)                      24.7995
IMPRESSIONS                                      155102.5
IMPRESSIONS CLICK-THROUGH RATE (%)                   2.43
AVERAGE VIEW S

In [106]:
# local picture

metric_agg = df_agg[[
    'VIDEO PUBLISH TIME',
    'COMMENTS ADDED', 
    'SHARES', 
    'DISLIKES', 
    'LIKES', 
    'SUBSCRIBERS GAINED', 
    'RPM (USD)', 
    'VIEWS', 
    'YOUR ESTIMATED REVENUE (USD)',
    'AVERAGE VIEW SECONDS', 
    'ENGAGEMENT RATIO', 
    'VIEW TO SUBSCRIBER RATIO',
]]

def metric_median(n):
    # 
    metric_date_n = metric_agg['VIDEO PUBLISH TIME'].max() - DateOffset(months=n)
    median_date_n = metric_agg[metric_agg['VIDEO PUBLISH TIME'] >= metric_date_n].median()
    
    return metric_date_n,median_date_n

metric_12mo, median_12mo = metric_median(12)
metric_6mo, median_6mo = metric_median(6)

In [107]:
len(metric_agg.columns)

12

In [108]:
median_6mo.index

Index(['VIDEO PUBLISH TIME', 'COMMENTS ADDED', 'SHARES', 'DISLIKES', 'LIKES',
       'SUBSCRIBERS GAINED', 'RPM (USD)', 'VIEWS',
       'YOUR ESTIMATED REVENUE (USD)', 'AVERAGE VIEW SECONDS',
       'ENGAGEMENT RATIO', 'VIEW TO SUBSCRIBER RATIO'],
      dtype='object')

In [109]:
median_6mo['VIEWS']

6062.0

In [110]:
for i in median_6mo.index:
    if i != 'VIDEO PUBLISH TIME':
        pass
        print(median_6mo[i] - median_12mo[i])
        #print((median_6mo[i] - median_12mo[i])/median_12mo[i])
    else:
        delta = median_6mo[i] - median_12mo[i]
        print(delta)
        print(delta.days)
        
        #st.metric(label = 'Duration', value = delta, delta=f"{delta} Days")

125 days 12:00:00
125
-7.5
-1.5
-0.5
-77.5
-21.0
0.024999999999999467
-1355.0
0.0
9.0
0.00682825754484416
6.723321127484667


In [111]:
df_agg_diff.dtypes

VIDEO                                         object
VIDEO TITLE                                   object
VIDEO PUBLISH TIME                    datetime64[ns]
COMMENTS ADDED                                 int64
SHARES                                         int64
DISLIKES                                       int64
LIKES                                          int64
SUBSCRIBERS LOST                               int64
SUBSCRIBERS GAINED                             int64
RPM (USD)                                    float64
CPM (USD)                                    float64
AVERAGE PERCENTAGE VIEWED (%)                float64
AVERAGE VIEW DURATION                 datetime64[ns]
VIEWS                                          int64
WATCH TIME (HOURS)                           float64
SUBSCRIBERS                                    int64
YOUR ESTIMATED REVENUE (USD)                 float64
IMPRESSIONS                                    int64
IMPRESSIONS CLICK-THROUGH RATE (%)           f

In [112]:
df_agg_diff.columns.tolist()

['VIDEO',
 'VIDEO TITLE',
 'VIDEO PUBLISH TIME',
 'COMMENTS ADDED',
 'SHARES',
 'DISLIKES',
 'LIKES',
 'SUBSCRIBERS LOST',
 'SUBSCRIBERS GAINED',
 'RPM (USD)',
 'CPM (USD)',
 'AVERAGE PERCENTAGE VIEWED (%)',
 'AVERAGE VIEW DURATION',
 'VIEWS',
 'WATCH TIME (HOURS)',
 'SUBSCRIBERS',
 'YOUR ESTIMATED REVENUE (USD)',
 'IMPRESSIONS',
 'IMPRESSIONS CLICK-THROUGH RATE (%)',
 'AVERAGE VIEW SECONDS',
 'ENGAGEMENT RATIO',
 'VIEW TO SUBSCRIBER RATIO',
 'VIEW TO SUBSCRIBER LOST RATIO']

In [113]:
df_agg_diff_final = df_agg_diff.loc[:,[
 'VIDEO',
 'VIDEO TITLE',
 'VIDEO PUBLISH TIME',
 'COMMENTS ADDED',
 'SHARES',
 'DISLIKES',
 'LIKES',
 'SUBSCRIBERS LOST',
 'SUBSCRIBERS GAINED',
 'VIEWS',
 'SUBSCRIBERS',
 'YOUR ESTIMATED REVENUE (USD)',
 'IMPRESSIONS',
 'IMPRESSIONS CLICK-THROUGH RATE (%)',
 'AVERAGE VIEW SECONDS',
 'ENGAGEMENT RATIO',
 'VIEW TO SUBSCRIBER RATIO',
 'VIEW TO SUBSCRIBER LOST RATIO']
]
df_agg_diff_final.head(1)

,VIDEO,VIDEO TITLE,VIDEO PUBLISH TIME,COMMENTS ADDED,SHARES,DISLIKES,LIKES,SUBSCRIBERS LOST,SUBSCRIBERS GAINED,VIEWS,SUBSCRIBERS,YOUR ESTIMATED REVENUE (USD),IMPRESSIONS,IMPRESSIONS CLICK-THROUGH RATE (%),AVERAGE VIEW SECONDS,ENGAGEMENT RATIO,VIEW TO SUBSCRIBER RATIO,VIEW TO SUBSCRIBER LOST RATIO
111,0jTtHYie3CU,Should You Be Excited About Web 3? (As a Data ...,2022-01-17,37,43,8,267,14,18,4383,4,16.549,65130,2.95,158,0.080995,243.5,313.071429


In [114]:
df_agg['PUBLISH DATE'] = df_agg.loc[:,'VIDEO PUBLISH TIME']

In [115]:
df_agg['VIDEO PUBLISH TIME'][9]

Timestamp('2020-07-01 00:00:00')

In [116]:
df_agg['PUBLISH DATE'][9]

Timestamp('2020-07-01 00:00:00')

In [117]:
df_agg.head(1)

,VIDEO,VIDEO TITLE,VIDEO PUBLISH TIME,COMMENTS ADDED,SHARES,DISLIKES,LIKES,SUBSCRIBERS LOST,SUBSCRIBERS GAINED,RPM (USD),...,WATCH TIME (HOURS),SUBSCRIBERS,YOUR ESTIMATED REVENUE (USD),IMPRESSIONS,IMPRESSIONS CLICK-THROUGH RATE (%),AVERAGE VIEW SECONDS,ENGAGEMENT RATIO,VIEW TO SUBSCRIBER RATIO,VIEW TO SUBSCRIBER LOST RATIO,PUBLISH DATE
111,0jTtHYie3CU,Should You Be Excited About Web 3? (As a Data ...,2022-01-17,37,43,8,267,14,18,4.055,...,192.5779,4,16.549,65130,2.95,158,0.080995,243.5,313.071429,2022-01-17


In [ ]:
# define functions

# load data
@st.cache_resource # loads this function once and doesn't reload everytime we reload our page
def load_data():

    # load data
    df_vid = pd.read_csv('resource/Video_Performance_Over_Time.csv')

    # dropping row Total to avoid issue with calculations
    df_agg = pd.read_csv('resource/Aggregated_Metrics_By_Video.csv').iloc[1:,:]

    df_agg_sub = pd.read_csv('resource/Aggregated_Metrics_By_Country_And_Subscriber_Status.csv')

    df_com = pd.read_csv('resource/All_Comments_Final.csv')

    # converting all columns to uppercase
    for df in [df_agg,df_vid,df_agg_sub, df_com]:
        df.columns = df.columns.str.upper()

    # remove \xad
    df_agg.columns = df_agg.columns.str.replace('\xad','')
    # convert date to datetype

    df_agg['VIDEO PUBLISH TIME'] = pd.to_datetime(df_agg['VIDEO PUBLISH TIME'], format='mixed')
        
    df_agg['AVERAGE VIEW DURATION'] = df_agg['AVERAGE VIEW DURATION'].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))

    # create new column for df_agg['AVERAGE VIEW SECONDS'
    df_agg['AVERAGE VIEW SECONDS'] = df_agg['AVERAGE VIEW DURATION'].apply(lambda x: x.second + x.minute * 60 + x.hour * 60 * 60)

    # engagement ration, every engagement a view could do divided by the number of viewrs
    df_agg['ENGAGEMENT RATIO'] = (df_agg['SHARES'] + df_agg['LIKES'] + df_agg['DISLIKES'] + df_agg['COMMENTS ADDED']) / df_agg['VIEWS'] 

    # ratio of views to subscribers gained
    df_agg['VIEW TO SUBSCRIBER RATIO'] = df_agg['VIEWS'] / df_agg['SUBSCRIBERS GAINED'] # how views does it take to gain a subscriber

    # ratio of views to subscribers lost, 
    df_agg['VIEW TO SUBSCRIBER LOST RATIO'] = df_agg['VIEWS'] / df_agg['SUBSCRIBERS LOST'] # how views does it take to lose a subscriber

    # sort data by 'VIDEO PUBLISH TIME'
    df_agg.sort_values(by = 'VIDEO PUBLISH TIME', ascending=False, inplace=True)

    # CONVERTING DATE to datetime
    df_vid['DATE'] = pd.to_datetime(df_vid['DATE'], format='mixed')

    df_com['DATE'] = pd.to_datetime(df_com['DATE'])

    # create dataframe
    return df_vid, df_agg, df_agg_sub, df_com

In [ ]:
df_vid, df_agg, df_agg_sub, df_com = load_data()

In [ ]:
# engineer data

# aggregated differential 

# create a copy of our dataframe
df_agg_diff = df_agg.copy()

# for the last 12 months, most recently date back to 12 months
metric_date_12mo = df_agg_diff['VIDEO PUBLISH TIME'].max() - DateOffset(months=12)

# dataframe from metric_date_12mo to df_agg_diff['VIDEO PUBLISH TIME'].max()
# that is, from 12 months early to current date

df_agg_diff_12mo = df_agg_diff[df_agg_diff['VIDEO PUBLISH TIME'] >= metric_date_12mo]

# median 
median_agg = df_agg_diff_12mo[df_agg_diff_12mo.columns[2:]].median()

## what metrics wil be relevant
## difference from baseline
## percent change

# build dashboard

# sidebar
add_sidebar = st.sidebar.selectbox("Aggregate or Individual Video", ("Aggregate Metrics", "Individual Video Analysis"))

# styling dataframe
def styling_positive(value, props):
    try:
        return props if value >= 0 else None
    except:
        pass


def styling_negative(value, props):
    try:
        return props if value < 0 else None
    except:
        pass

In [159]:
## local picture
if add_sidebar == "Aggregate Metrics":

    metric_agg = df_agg[[
        'VIDEO PUBLISH TIME',
        'COMMENTS ADDED', 
        'SHARES', 
        'DISLIKES', 
        'LIKES', 
        'SUBSCRIBERS GAINED', 
        'RPM (USD)', 
        'VIEWS', 
        'YOUR ESTIMATED REVENUE (USD)',
        'AVERAGE VIEW SECONDS', 
        'ENGAGEMENT RATIO', 
        'VIEW TO SUBSCRIBER RATIO',
    ]]  

    def metric_median(n):
        # date range
        metric_date_n = metric_agg['VIDEO PUBLISH TIME'].max() - DateOffset(months=n)
        median_date_n = metric_agg[metric_agg['VIDEO PUBLISH TIME'] >= metric_date_n].median()
        
        return metric_date_n,median_date_n

    metric_12mo, median_12mo = metric_median(12)
    metric_6mo, median_6mo = metric_median(6)

    col1, col2, col3,col4,col5,col6 = st.columns(6)
    columns = [col1, col2, col3,col4,col5,col6]

    count = 0
    for i in median_6mo.index:
        with columns[count]:
            if i != 'VIDEO PUBLISH TIME':
                delta = (median_6mo[i] - median_12mo[i])/median_12mo[i]
                st.metric(label = i, value =round(median_6mo[i]), delta="{:.2%}".format(delta))
            else:
                delta = median_6mo[i] - median_12mo[i]
                st.metric(label = 'Duration', value = delta.days, delta=f"{(delta//30)} Months")
            count += 1
            if count >= 6:
                count = 0


    df_agg_diff_final = df_agg_diff.loc[:,[
    'VIDEO',
    'VIDEO TITLE',
    'VIDEO PUBLISH TIME',
    'COMMENTS ADDED',
    'SHARES',
    'DISLIKES',
    'LIKES',
    'SUBSCRIBERS LOST',
    'SUBSCRIBERS GAINED',
    'VIEWS',
    'SUBSCRIBERS',
    'YOUR ESTIMATED REVENUE (USD)',
    'IMPRESSIONS',
    'IMPRESSIONS CLICK-THROUGH RATE (%)',
    'AVERAGE VIEW DURATION',
    'AVERAGE VIEW SECONDS',
    'ENGAGEMENT RATIO',
    'VIEW TO SUBSCRIBER RATIO',
    'VIEW TO SUBSCRIBER LOST RATIO']
    ]
    
    # extract only date
    df_agg_diff_final['VIDEO PUBLISH TIME'] = df_agg_diff_final['VIDEO PUBLISH TIME'].dt.date

    # rename column

    df_agg_diff_final.rename(columns={'VIDEO PUBLISH TIME': 'PUBLISH DATE'}, inplace=True)

    # extracting time
    df_agg_diff_final['AVERAGE VIEW DURATION'] = df_agg_diff_final['AVERAGE VIEW DURATION'].dt.time


    # formating each data to percentage
    

    st.dataframe(df_agg_diff_final.style.hide().applymap(styling_positive, props = 'color:green;').applymap(styling_negative, props = 'color:red;'))

elif add_sidebar =="Individual Video Analysis":
    st.write('Ind')


## individual video

# improvement

# styling

/tmp/ipykernel_55684/731254956.py:187: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  st.dataframe(df_agg_diff_final.style.hide().applymap(styling_positive, props = 'color:green;').applymap(styling_negative, props = 'color:red;'))
